## Load Libraries

In [1]:
import pandas as pd
import numpy as np
from pandas import Timestamp
import os
from datetime import datetime, timedelta

## Read the JSON files from directory

In [2]:
# Read the directories with the data and save file_names in two list
path_to_weather = 'python_data/weather_data/'

json_files_weather = [single_json for single_json in os.listdir(path_to_weather) if single_json.endswith('.json')]

In [3]:
json_files_weather[:5]

['Weather_Data_2017_06_05_10_16_12.json',
 'Weather_Data_2017_06_06_10_25_31.json',
 'Weather_Data_2017_06_06_10_26_02.json',
 'Weather_Data_2017_06_06_10_26_17.json',
 'Weather_Data_2017_06_06_10_27_28.json']

In [4]:
#Check a random json to see how it looks like
with open(path_to_weather+json_files_weather[0]) as json_data:
    d = json.load(json_data)
d

{u'base': u'stations',
 u'clouds': {u'all': 20},
 u'cod': 200,
 u'coord': {u'lat': 55.67, u'lon': 12.59},
 u'dt': 1496649000,
 u'id': 2618424,
 u'main': {u'humidity': 67,
  u'pressure': 1013,
  u'temp': 287.9,
  u'temp_max': 288.15,
  u'temp_min': 287.15},
 u'name': u'K\xf8benhavns Kommune',
 u'sys': {u'country': u'DK',
  u'id': 5245,
  u'message': 0.0035,
  u'sunrise': 1496629792,
  u'sunset': 1496692037,
  u'type': 1},
 u'visibility': 10000,
 u'weather': [{u'description': u'few clouds',
   u'icon': u'02d',
   u'id': 801,
   u'main': u'Clouds'}],
 u'wind': {u'deg': 260, u'speed': 5.1}}

In [73]:
d['weather'][0]['icon']

u'02d'

## Useful Functions

In [5]:
# Create a function that gets the date from filename title
def getDate(s):
    first_split = s.split('_')[2]
    second_split = s.split('_')[3]
    third_split = s.split('_')[4]
    return first_split + '-' + second_split + '-' + third_split

In [6]:
# Create a function that gets the time from filename title
def getTime(s):
    first_split = s.split('_')[5]
    second_split = s.split('_')[6]
    third_split = s.split('_')[7][:-5]
    return first_split + ':' + second_split + ':' + third_split

In [7]:
# Create a good format for the Date column
def dateConverter(s):
    # Set date formats
    time_format = "%Y-%m-%d"

    # Convert from str to datetime
    converted = datetime.strptime(s,time_format)
    
    return converted

In [8]:
testDate = getDate(json_files_weather[0])

In [9]:
testTime = getTime(json_files_weather[0])

In [10]:
testDate

'2017-06-05'

In [12]:
# Initiate a new dataframe
df = pd.DataFrame()

# Parse json weather files data
for i,filename in enumerate(json_files_weather):
    with open(path_to_weather+filename) as json_data:
        d = json.load(json_data)
        df.set_value(i,'date',dateConverter(getDate(filename)))
        df.set_value(i,'time',getTime(filename))
        df.set_value(i,'windSpeed',d['wind']['speed'] * 3.6)
        df.set_value(i,'windDeg',d['wind']['deg'])
        df.set_value(i,'humidity%',d['main']['humidity'])
        df.set_value(i,'tempC',d['main']['temp'] - 272.15)
        df.set_value(i,'icon',d['weather'][0]['icon'])
        df.set_value(i,'desc',d['weather'][0]['description'])
        df.set_value(i,'lon', d['coord']['lon'])
        df.set_value(i,'lat', d['coord']['lat'])

In [79]:
df

,date,time,windSpeed,windDeg,humidity%,tempC,icon,desc,lon,lat
0,2017-06-05,10:16:12,18.36,260.0,67.0,15.75,02d,few clouds,12.59,55.67
1,2017-06-06,10:25:31,25.92,140.0,72.0,19.02,02d,few clouds,12.59,55.69
2,2017-06-06,10:26:02,25.92,140.0,72.0,19.02,02d,few clouds,12.59,55.69
3,2017-06-06,10:26:17,25.92,140.0,72.0,19.02,02d,few clouds,12.59,55.69
4,2017-06-06,10:27:28,25.92,140.0,72.0,19.02,02d,few clouds,12.59,55.69
5,2017-06-06,10:29:28,25.92,140.0,72.0,19.02,02d,few clouds,12.59,55.69
6,2017-06-06,10:33:28,25.92,140.0,72.0,19.02,02d,few clouds,12.59,55.69
7,2017-06-06,10:41:31,25.92,140.0,72.0,19.02,02d,few clouds,12.59,55.69
8,2017-06-06,10:57:33,25.92,130.0,63.0,19.41,04d,broken clouds,12.59,55.69
9,2017-06-06,11:32:39,24.12,130.0,63.0,19.82,04d,broken clouds,12.59,55.69
